In [1]:
import os
from datetime import datetime 
import pandas as pd
from geopy.distance import geodesic
import download_data

In [2]:
# Load data. Download if haven't already

data_url = "https://divvy-tripdata.s3.amazonaws.com/index.html"
data_dir = os.getcwd()+'/Data'
filename = 'raw_data.csv'
file_location = os.path.join(data_dir,filename)

if not os.path.exists(file_location):
    download_data.download_data(data_dir,data_url)
    filenames = os.listdir(data_dir).copy()
    file_paths = [os.path.join(data_dir,f) for f in filenames if not f.startswith('.')]
    raw_data = pd.concat([pd.read_csv(filepath) for filepath in file_paths])
    raw_data.to_csv(file_location)
else:
    raw_data = pd.read_csv(file_location)

/Users/tylerbell/opt/anaconda3/envs/myenv/lib/python3.8/site-packages/IPython/core/interactiveshell.py:3165: DtypeWarning: Columns (6,8) have mixed types.Specify dtype option on import or set low_memory=False.
  has_raised = await self.run_ast_nodes(code_ast.body, cell_name,


In [3]:
# Drop rows missing crucial information

num_rows = len(raw_data)
print('In total, there are {} observations in the original dataset'.format(num_rows))
key_feats = ['ride_id','rideable_type','started_at','ended_at','start_lat','start_lng','end_lat','end_lng','member_casual', 'start_station_id','end_station_id']
raw_data.dropna(subset = key_feats, inplace = True)
print('\n{} rows were dropped due to missing info'.format(num_rows-len(raw_data)))

In total, there are 5378834 observations in the original dataset

886106 rows were dropped due to missing info


In [4]:
# Handling datetime

str_to_dt = lambda s: datetime.strptime(s, '%Y-%m-%d %H:%M:%S')
raw_data['start_dt'] = raw_data['started_at'].transform(str_to_dt)
raw_data['end_dt'] = raw_data['ended_at'].transform(str_to_dt)

# Extract granular data from datetime

raw_data['month'] = raw_data['start_dt'].apply(lambda x: x.month)
raw_data['day'] = raw_data['start_dt'].apply(lambda x: x.weekday())
raw_data['hour'] = raw_data['start_dt'].apply(lambda x: x.hour)

# Calculate duration of trip

start_end = zip(raw_data['start_dt'],raw_data['end_dt'])
raw_data['delta(seconds)'] = [(end-start).total_seconds() for start,end in start_end]
raw_data['delta(minutes)'] = raw_data['delta(seconds)']/60

# Remove non postive or over 24h durations

non_positive_idx = raw_data['delta(minutes)'] <= 0
over_24hr_idx = raw_data['delta(minutes)'] > int(24*60)
raw_data = raw_data[~(non_positive_idx|over_24hr_idx)]
num_rows_dropped = sum((non_positive_idx|over_24hr_idx))

print('{} rows were dropped due to anomalous durations'.format(num_rows_dropped))

2713 rows were dropped due to anomalous durations


In [5]:
# Quantize time of day

bin_interval = pd.IntervalIndex.from_tuples
bins = bin_interval([(-1, 5), (5, 11), (11, 17), (17,23)])
labels = ["Small Hours", "Morning", "Afternoon", "Evening"]
bin_label = dict(zip(bins,labels))
raw_data['time_of_day'] = pd.cut(raw_data['hour'],bins).map(bin_label)

# Map the month to season

bins = bin_interval([(-1, 2), (2, 5), (5, 8), (8,11)])
labels = ["Winter", "Spring", "Summer", "Fall"]
bin_label = dict(zip(bins,labels))
raw_data['season'] = pd.cut(raw_data['month'].apply(lambda x: x%12),bins).map(bin_label)

# Create weekend boolean

raw_data['is_weekend'] = raw_data['day'] >= 5

# Categorize trip length

bins = bin_interval([(0,10),(10,30),(30,120),(120,24*60)])
labels = ["Under 10mins", "Under 30mins", "Under 2hours", "More than 2hours"]
bin_label = dict(zip(bins,labels))
raw_data["trip_length"] = pd.cut(raw_data['delta(minutes)'],bins).map(bin_label)

In [6]:
# Handling geolocation data

start_coords = zip(raw_data['start_lat'],raw_data['start_lng'])
end_coords = zip(raw_data['end_lat'],raw_data['end_lng'])

raw_data['start_coord'] = [(lat,lng) for lat,lng in start_coords]
raw_data['end_coord'] = [(lat,lng) for lat,lng in end_coords]

start_end_coord = zip(raw_data['start_coord'], raw_data['end_coord'])

raw_data['delta(meters)'] = [geodesic(start,end).meters for start,end in start_end_coord]

In [7]:
# Calculating average velocity

delta = zip(raw_data['delta(meters)'],raw_data['delta(seconds)'])
raw_data['avg_velocity(m/s)'] = [m/s for m,s in delta]

# Remove rows with bikes going over max speed of electric bike (20mph)

mph_ms_conversion = 0.44704
max_speed = 20*mph_ms_conversion

over_limit = raw_data['avg_velocity(m/s)'] > max_speed

raw_data = raw_data[~over_limit]
v_dropped = sum(over_limit)

print('{} rows dropped because average velocity greater than max speed'.format(v_dropped))

294 rows dropped because average velocity greater than max speed


In [8]:
ride_cols = ['ride_id','rideable_type', 'member_casual', 'start_station_id', 'end_station_id']
time_cols = ['month','season','day','is_weekend', 'hour', 'time_of_day']
delta_cols = ['delta(meters)', 'delta(seconds)','trip_length','avg_velocity(m/s)']

cols = [*ride_cols,*time_cols,*delta_cols]

data = raw_data[cols]
data.to_csv(os.path.join(data_dir,'clean_data.csv'), index=False)

print('In total our cleaned data has {} observations with {} features'.format(len(data),len(cols)))

In total our cleaned data has 4489721 observations with 15 features
